In [1]:
# we scraper
from bs4 import BeautifulSoup

# request
import requests

# io
from io import BytesIO

# date parser and sleep
from dateutil.parser import parse
import time

# web driver
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc

# reeading and exporting to excel files
import pandas as pd


# looping process monitoring
from tqdm import tqdm
import random

In [2]:
# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

s = slice(11)

In [3]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("SEALAND")

In [4]:
len(bl_list)

37

In [5]:
bl_list

['227145963',
 '227927058',
 '228000484',
 '228698779',
 '228667042',
 '228487087',
 '228495628',
 '228528205',
 '228151079',
 '228326741',
 '228329054',
 '228776329',
 '228107228',
 '228404249',
 '228876620',
 '228883601',
 '228877614',
 '229154632',
 '229327088',
 '232450086',
 '232754943',
 '232849365',
 '232841994',
 '232845045',
 '232815698',
 '233105679',
 '232927940',
 '233304875',
 '233352740',
 '233589084',
 '233361678',
 '233477129',
 '233407826',
 '233060834',
 '233110018',
 '233124829',
 '233222351']

In [6]:
dict_hasil = {}
current_dict = {}
list_dict = []
gagal = []

In [20]:
bl_list = ['232450086',
 '232754943',
 '232849365',
 '232841994',
 '232845045',
 '232815698',
 '233105679',
 '232927940',
 '233304875',
 '233352740',
 '233589084',
 '233361678',
 '233477129',
 '233407826',
 '233060834',
 '233110018',
 '233124829',
 '233222351']

**FIX**


In [23]:
dict_hasil = {}
current_dict = {}
list_dict = []
gagal = []

# web scripting
options = Options()
options.add_argument("--window-size=1920,1280")
driver = uc.Chrome(use_subprocess=True)
driver.get('https://www.maersk.com/tracking/' + bl_list[random.randint(0, len(bl_list)-1)])

# wait to load all
time.sleep(2)

#click allow all
click_allowall = driver.find_element(By.XPATH,'//button[text()="Allow all"]')
click_allowall.click()

for i, bls in enumerate(tqdm(bl_list)):
    try:
        # access the web
        driver.get('https://www.maersk.com/tracking/' + bl_list[i])
        # sleep wait
        title = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='maersk-app']/div/div/div/dl")))
        time.sleep(3)
        
        # finding how many hide details in 1 page refering to how many container number in 1 bl
        soup1 = BeautifulSoup(driver.page_source, 'lxml')
        containers = soup1.find_all('div', {"data-test": "container"})
        number_of_ctr = len(containers)
        
        print(bls, "HAVE ", number_of_ctr, " CTR")
        # print(dict_hasil)

        if number_of_ctr > 1:
            titles = title.text.split('\n')
            titles.remove('View Shipment Details')
            print(titles)
            for a, item in enumerate(titles):
                #
                if a%2==0 :
                    case_judul = {titles[a]:titles[a+1]}
                    dict_hasil.update(case_judul)
            
            print("Hasil dari Append Judul")
            print(dict_hasil)
            if driver.find_element(By.CLASS_NAME, "track__feedback"):
                x=4
            else:
                x=3
            # clicking all view more
            for c in range(number_of_ctr):
                try:
                    print("clicking div no :",  x)
                    show_details2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/div/div/div/div/div[{}]/dl/button'.format(x))))  
                    # /html/body/main/div/div/div/div/div[3]/dl/button/span
                    show_details_check = driver.find_element(By.XPATH, "/html/body/main/div/div/div/div/div[{}]/dl/button/span".format(x))
                    if show_details_check.text == 'Show details':
                        show_details2.click()
                    print(show_details_check.text)
                    
                    time.sleep(1)
                except Exception as e:
                    x=x+1

            time.sleep(2)
            # taking the whole page's html
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            
            table = soup2.find_all('div', {"data-test": "container"})
            milestones = []
            for tag in table:
                for ctri1, ctr_num1  in enumerate(tag.find_all('strong', {"data-test":'container-details-value'})):
                    temp_ctr_num1 = ctr_num1
                    milestones.append("Container Number")
                    milestones.append(temp_ctr_num1.text)
                for m, li_milestone in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-complete'})):
                    span_history = li_milestone.find_all("span")
                    for c1, city1 in enumerate(li_milestone.find_all("strong")):
                        temp1 = city1.text
                    for milestone_span in span_history:
                        if is_date(milestone_span.text):
                            milestones.append(milestone_span.text[s])
                        else:
                            milestones.append(temp1 +" "+ milestone_span.text)
                            
                for n, li_expected in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-future'})):
                    spans = li_expected.find_all("span")
                    for c2, city2 in enumerate(li_expected.find_all("strong")):
                        temp2 = city2.text
                    for expected_span in spans:
                        if is_date(expected_span.text):
                            milestones.append(expected_span.text[s])
                        else:
                            milestones.append(temp2 + " ESTIMATED " + expected_span.text)
                 
                for panjang in range(0, len(milestones), 2):
                    key = milestones[panjang]
                    value = milestones[panjang + 1]

                    if key == 'Container Number':
                        if current_dict:
                            list_dict.append(current_dict.copy())
                        current_dict = {'Container Number': value}
                    else:
                        current_dict[key] = value

                    current_dict.update(dict_hasil) 
                print(current_dict)
            list_dict.append(current_dict)
            current_dict = {}
            dict_hasil={}
            print(bls, ", completed!!")
        else:
            time.sleep(1)
            
            titles = title.text.split('\n')
            titles.remove('View Shipment Details')
            print(titles)
            for a, item in enumerate(titles):
                if a%2==0 :
                    case_judul = {titles[a]:titles[a+1]}
                    dict_hasil.update(case_judul)
            
            time.sleep(2)
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            
            table = soup2.find_all('div', {"data-test": "container"})
            milestones = []
            for tag in table:
                for ctri1, ctr_num1  in enumerate(tag.find_all('strong', {"data-test":'container-details-value'})):
                    temp_ctr_num1 = ctr_num1
                    milestones.append("Container Number")
                    milestones.append(temp_ctr_num1.text)

                for m, li_milestone in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-complete'})):
                    span_history = li_milestone.find_all("span")
                    for c1, city1 in enumerate(li_milestone.find_all("strong")):
                        temp1 = city1.text
                    for milestone_span in span_history:
                        if is_date(milestone_span.text):
                            milestones.append(milestone_span.text[s])
                        else:
                            milestones.append(temp1 +" "+ milestone_span.text)
                            
                for n, li_expected in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-future'})):
                    spans = li_expected.find_all("span")
                    for c2, city2 in enumerate(li_expected.find_all("strong")):
                        temp2 = city2.text
                    for expected_span in spans:
                        if is_date(expected_span.text):
                            milestones.append(expected_span.text[s])
                        else:
                            milestones.append(temp2 + " ESTIMATED " + expected_span.text)
            
            for t, milestone in enumerate(milestones):
                if milestone == "Container Number":
                    case_mlstn = {milestones[t]:milestones[t+1]}
                    current_dict.update(case_mlstn)
                elif t%2==0 :
                    case_mlstn = {milestones[t]:milestones[t+1]}
                    current_dict.update(case_mlstn)
                    
            current_dict.update(dict_hasil) 
            print(current_dict)
            list_dict.append(current_dict)
            current_dict = {}
            dict_hasil={}
        
        unique_list = []
        for d in list_dict:
            if d not in unique_list:
                unique_list.append(d)
                
    except Exception as e:
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)

  0%|          | 0/18 [00:00<?, ?it/s]

232450086 HAVE  7  CTR
['Bill of Lading number', '232450086', 'From', 'Jakarta', 'To', 'Xingang']
Hasil dari Append Judul
{'Bill of Lading number': '232450086', 'From': 'Jakarta', 'To': 'Xingang'}
clicking div no : 4
Hide details
clicking div no : 4
Hide details
clicking div no : 4
Hide details
clicking div no : 4
Hide details
clicking div no : 4
Hide details
clicking div no : 4
Hide details
clicking div no : 4
Hide details


  6%|▌         | 1/18 [00:18<05:22, 18.96s/it]

{'Container Number': 'TRHU4064403', 'Bill of Lading number': '232450086', 'From': 'Jakarta', 'To': 'Xingang', 'Jakarta Gate outEmpty': '24 Oct 2023', 'Jakarta Gate in': '26 Oct 2023', 'Jakarta Load': '29 Oct 2023', 'Jakarta Vessel departure': '30 Oct 2023', 'Xingang Vessel arrival': '18 Nov 2023', 'Xingang Discharge': '18 Nov 2023', 'Xingang Gate out for delivery': '20 Nov 2023'}
{'Container Number': 'TCLU1608884', 'Bill of Lading number': '232450086', 'From': 'Jakarta', 'To': 'Xingang', 'Jakarta Gate outEmpty': '24 Oct 2023', 'Jakarta Gate in': '26 Oct 2023', 'Jakarta Load': '30 Oct 2023', 'Jakarta Vessel departure': '30 Oct 2023', 'Xingang Vessel arrival': '18 Nov 2023', 'Xingang Discharge': '18 Nov 2023', 'Xingang Gate out for delivery': '20 Nov 2023'}
{'Container Number': 'TCNU8542662', 'Bill of Lading number': '232450086', 'From': 'Jakarta', 'To': 'Xingang'}
{'Container Number': 'MSKU0835371', 'Bill of Lading number': '232450086', 'From': 'Jakarta', 'To': 'Xingang'}
{'Container Nu

 11%|█         | 2/18 [00:47<06:36, 24.80s/it]

...

{'Container Number': 'HASU5169807', 'Jakarta Gate outEmpty': '09 Nov 2023', 'Jakarta Gate in': '12 Nov 2023', 'Jakarta Load': '17 Nov 2023', 'Jakarta Vessel departure': '17 Nov 2023', 'Tanjung Pelepas Vessel arrival': '19 Nov 2023', 'Tanjung Pelepas Discharge': '19 Nov 2023', 'Tanjung Pelepas Gate out for delivery': '24 Nov 2023', 'Pasir Gudang Empty container return': '28 Nov 2023', 'Bill of Lading number': '233110018', 'From': 'Jakarta', 'To': 'Tanjung Pelepas'}
233124829 HAVE  1  CTR
['Bill of Lading number', '233124829', 'From', 'Jakarta', 'To', 'Kuching']


 94%|█████████▍| 17/18 [03:59<00:11, 11.86s/it]

{'Container Number': 'MRSU5026827', 'Balikpapan Load': '02 Nov 2023', 'Balikpapan Gate out for delivery': '02 Nov 2023', 'Jakarta Gate outEmpty': '10 Nov 2023', 'Jakarta Gate in': '13 Nov 2023', 'Jakarta Load': '18 Nov 2023', 'Jakarta Vessel departure': '18 Nov 2023', 'Tanjung Pelepas Vessel arrival': '21 Nov 2023', 'Tanjung Pelepas Discharge': '21 Nov 2023', 'Tanjung Pelepas Load': '21 Nov 2023', 'Tanjung Pelepas Vessel departure': '22 Nov 2023', 'Kuching Vessel arrival': '25 Nov 2023', 'Kuching Discharge': '27 Nov 2023', 'Kuching Gate out for delivery': '28 Nov 2023', 'Bill of Lading number': '233124829', 'From': 'Jakarta', 'To': 'Kuching'}
233222351 HAVE  1  CTR
['Bill of Lading number', '233222351', 'From', 'Jakarta', 'To', 'Kuching']


100%|██████████| 18/18 [04:10<00:00, 13.92s/it]

{'Container Number': 'HASU5186424', 'Jakarta Gate outEmpty': '11 Nov 2023', 'Jakarta Gate in': '13 Nov 2023', 'Jakarta Load': '18 Nov 2023', 'Jakarta Vessel departure': '18 Nov 2023', 'Tanjung Pelepas Vessel arrival': '21 Nov 2023', 'Tanjung Pelepas Discharge': '21 Nov 2023', 'Tanjung Pelepas Load': '28 Nov 2023', 'Tanjung Pelepas Vessel departure': '29 Nov 2023', 'Kuching Vessel arrival': '30 Nov 2023', 'Bill of Lading number': '233222351', 'From': 'Jakarta', 'To': 'Kuching'}


In [24]:
unique_list

[{'Container Number': 'TRHU4064403',
  'Bill of Lading number': '232450086',
  'From': 'Jakarta',
  'To': 'Xingang',
  'Jakarta Gate outEmpty': '24 Oct 2023',
  'Jakarta Gate in': '26 Oct 2023',
  'Jakarta Load': '29 Oct 2023',
  'Jakarta Vessel departure': '30 Oct 2023',
  'Xingang Vessel arrival': '18 Nov 2023',
  'Xingang Discharge': '18 Nov 2023',
  'Xingang Gate out for delivery': '20 Nov 2023'},
 {'Container Number': 'TCLU1608884',
  'Bill of Lading number': '232450086',
  'From': 'Jakarta',
  'To': 'Xingang',
  'Jakarta Gate outEmpty': '24 Oct 2023',
  'Jakarta Gate in': '26 Oct 2023',
  'Jakarta Load': '30 Oct 2023',
  'Jakarta Vessel departure': '30 Oct 2023',
  'Xingang Vessel arrival': '18 Nov 2023',
  'Xingang Discharge': '18 Nov 2023',
  'Xingang Gate out for delivery': '20 Nov 2023'},
 {'Container Number': 'TCNU8542662',
  'Bill of Lading number': '232450086',
  'From': 'Jakarta',
  'To': 'Xingang'},
 {'Container Number': 'MSKU0835371',
  'Bill of Lading number': '2324500

In [25]:
filtered_dict_list =[]
for dict_milestone in unique_list:
    # print(dict_milestone)
    kw = ['Bill of Lading number', 'Container Number', 'From', 'To', dict_milestone['From'], dict_milestone['To']]
    dict_key = []
    for keys in dict_milestone:
        dict_key.append(keys)
    # print("BEFORE", dict_key)
    new_list = [] 
    
    for key in dict_key:
        if any(substring in key for substring in kw):
            new_list.append(key)
    # print("AFTER",new_list)
    dict_milestone = {key: dict_milestone[key] for key in new_list}
    filtered_dict_list.append(dict_milestone)

In [26]:
filtered_dict_list

[{'Container Number': 'TRHU4064403',
  'Bill of Lading number': '232450086',
  'From': 'Jakarta',
  'To': 'Xingang',
  'Jakarta Gate outEmpty': '24 Oct 2023',
  'Jakarta Gate in': '26 Oct 2023',
  'Jakarta Load': '29 Oct 2023',
  'Jakarta Vessel departure': '30 Oct 2023',
  'Xingang Vessel arrival': '18 Nov 2023',
  'Xingang Discharge': '18 Nov 2023',
  'Xingang Gate out for delivery': '20 Nov 2023'},
 {'Container Number': 'TCLU1608884',
  'Bill of Lading number': '232450086',
  'From': 'Jakarta',
  'To': 'Xingang',
  'Jakarta Gate outEmpty': '24 Oct 2023',
  'Jakarta Gate in': '26 Oct 2023',
  'Jakarta Load': '30 Oct 2023',
  'Jakarta Vessel departure': '30 Oct 2023',
  'Xingang Vessel arrival': '18 Nov 2023',
  'Xingang Discharge': '18 Nov 2023',
  'Xingang Gate out for delivery': '20 Nov 2023'},
 {'Container Number': 'TCNU8542662',
  'Bill of Lading number': '232450086',
  'From': 'Jakarta',
  'To': 'Xingang'},
 {'Container Number': 'MSKU0835371',
  'Bill of Lading number': '2324500

In [27]:
list_of_dict_fix = []
for filter_dict in filtered_dict_list:
    replacement_mapping = {
        filter_dict["From"]: 'Origin',
        filter_dict["To"]: 'Destination'
    }

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            key = key.replace(old_key, new_key)
        updated_dict[key] = value

    list_of_dict_fix.append(updated_dict)

In [28]:
list_of_dict_fix

[{'Container Number': 'TRHU4064403',
  'Bill of Lading number': '232450086',
  'From': 'Jakarta',
  'To': 'Xingang',
  'Origin Gate outEmpty': '24 Oct 2023',
  'Origin Gate in': '26 Oct 2023',
  'Origin Load': '29 Oct 2023',
  'Origin Vessel departure': '30 Oct 2023',
  'Destination Vessel arrival': '18 Nov 2023',
  'Destination Discharge': '18 Nov 2023',
  'Destination Gate out for delivery': '20 Nov 2023'},
 {'Container Number': 'TCLU1608884',
  'Bill of Lading number': '232450086',
  'From': 'Jakarta',
  'To': 'Xingang',
  'Origin Gate outEmpty': '24 Oct 2023',
  'Origin Gate in': '26 Oct 2023',
  'Origin Load': '30 Oct 2023',
  'Origin Vessel departure': '30 Oct 2023',
  'Destination Vessel arrival': '18 Nov 2023',
  'Destination Discharge': '18 Nov 2023',
  'Destination Gate out for delivery': '20 Nov 2023'},
 {'Container Number': 'TCNU8542662',
  'Bill of Lading number': '232450086',
  'From': 'Jakarta',
  'To': 'Xingang'},
 {'Container Number': 'MSKU0835371',
  'Bill of Lading n

In [29]:
list_of_dict_fix2 = []

for filter_dict in list_of_dict_fix:
    replacement_mapping = {
        "Bill of Lading number": "BL Number",
        "Origin Vessel departure": 'ATD',
        "Destination Discharge": 'ATA',
        "Destination Gate out": 'Container Release',
        "Destination Empty container return": 'Container Return',
    }

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            if old_key in key:
                key = new_key

        updated_dict[key] = value

        if is_date(value):
            input_date = datetime.strptime(value, '%d %b %Y')
            updated_dict[key] = input_date.strftime("%Y-%m-%d")

    updated_dict["Liners"] = "SEALAND"

    list_of_dict_fix2.append(updated_dict)

In [30]:
list_of_dict_fix2

[{'Container Number': 'TRHU4064403',
  'BL Number': '232450086',
  'From': 'Jakarta',
  'To': 'Xingang',
  'Origin Gate outEmpty': '2023-10-24',
  'Origin Gate in': '2023-10-26',
  'Origin Load': '2023-10-29',
  'ATD': '2023-10-30',
  'Destination Vessel arrival': '2023-11-18',
  'ATA': '2023-11-18',
  'Container Release': '2023-11-20',
  'Liners': 'SEALAND'},
 {'Container Number': 'TCLU1608884',
  'BL Number': '232450086',
  'From': 'Jakarta',
  'To': 'Xingang',
  'Origin Gate outEmpty': '2023-10-24',
  'Origin Gate in': '2023-10-26',
  'Origin Load': '2023-10-30',
  'ATD': '2023-10-30',
  'Destination Vessel arrival': '2023-11-18',
  'ATA': '2023-11-18',
  'Container Release': '2023-11-20',
  'Liners': 'SEALAND'},
 {'Container Number': 'TCNU8542662',
  'BL Number': '232450086',
  'From': 'Jakarta',
  'To': 'Xingang',
  'Liners': 'SEALAND'},
 {'Container Number': 'MSKU0835371',
  'BL Number': '232450086',
  'From': 'Jakarta',
  'To': 'Xingang',
  'Liners': 'SEALAND'},
 {'Container Num

In [31]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(list_of_dict_fix2)

Today's Collection Name ===>  all_tracking_Dec-01-2023
Today's Collection Has Been Made
Inserting Many Complete!!


In [33]:
gagal

[]

In [63]:
df = pd.DataFrame.from_dict(list_of_dict_fix2)
df.to_excel('sealand_maintained.xlsx')